In [1]:
#필요한 라이브러리들 import
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import dataset
import model
import torch.nn.functional as F

c:\Users\USER\anaconda3\envs\fskd\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 각종 path및 하이퍼 파라미터 설정
data_path = 'C:\\Users\\USER\\Desktop\\GSH_CRP\\codes\\rock_sci_paper\\data\\ro_sci_pa'
save_path = 'C:\\Users\\USER\\Desktop\\GSH_CRP\\codes\\rock_sci_paper\\model_para'
epochs = 100
batch_size = 16
learning_rate = 0.01
seed = 2024
mode = 'hr'

In [3]:
# seed 설정
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# device 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
# transform 설정
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256,256)),
    transforms.CenterCrop((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation((-90, 90)),
    transforms.RandomAffine((-90, 90), translate=(0.2,0.2)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

In [5]:
# dataset 설정
datasets = dataset.RockScissorsPaper(
    transform=transform,
    path = data_path
)
num_data = len(datasets)
num_train = int(num_data*0.6)
num_val = int(num_data*0.2)
num_test = num_data - num_train - num_val

train_data, val_data, test_data = torch.utils.data.random_split(datasets, [num_train, num_val, num_test])


In [6]:
# dataloader 설정
trainloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valloader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [7]:
# 모델, 손실함수, 옵티마이저 설정
model = model.ResNet18(num_classes=3)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
def train(epoch):
    print('\nEpoch: %d'%epoch)
    # model train mode로 전환
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    total = 0
    for (inputs, labels) in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        if mode=='lr':
            h,w = inputs.shape[-2], inputs.shape[-1]
            lr_inputs = F.interpolate(inputs, (h//64, w//64))
            lr_inputs = F.interpolate(lr_inputs, (h,w))
            outputs, _, _, _, _ = model(lr_inputs)
        else:
            outputs, _, _, _, _ = model(inputs)
            
        _, pred = torch.max(outputs, 1)
        total += outputs.size(0)
        running_acc += (pred == labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    total_loss = running_loss / len(trainloader)
    total_acc = 100 * running_acc / total
    print(f'Train epoch : {epoch} loss : {total_loss} Acc : {total_acc}%')

In [9]:
def test(epoch, loader, mode='val'):
    print('\nEpoch: %d'%epoch)
    # model eval mode로 전환
    model.eval()
    running_loss = 0.0
    running_acc = 0.0
    total = 0
    label_dict = {0:0, 1:0, 2:0}
    global BEST_SCORE
    with torch.no_grad():
        for (inputs, labels) in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            if mode=='lr':
                h,w = inputs.shape[-2], inputs.shape[-1]
                lr_inputs = F.interpolate(inputs, (h//32, w//32))
                lr_inputs = F.interpolate(lr_inputs, (h,w))
                outputs, _, _, _, _ = model(lr_inputs)
            else:
                outputs, _, _, _, _ = model(inputs)

            _, pred = torch.max(outputs, 1)
            for label in labels:
                label_dict[label.item()] += 1
            total += outputs.size(0)
            running_acc += (pred == labels).sum().item()
            loss = criterion(outputs, labels)
            running_loss += loss.item()
        total_loss = running_loss / len(loader)
        total_acc = 100 * running_acc / total
        print(label_dict)
        if total_acc > BEST_SCORE and not mode=='test':
            path = os.path.join(save_path, f'teacher.pth')
            torch.save(model.state_dict(), path)
            BEST_SCORE = total_acc
        print(f'Test epoch : {epoch} loss : {total_loss} Acc : {total_acc}%')

In [10]:
# 모델 학습 및 평가
BEST_SCORE = 0
for epoch in range(epochs):
    train(epoch)
    test(epoch, valloader)
    print(BEST_SCORE)


Epoch: 0
Train epoch : 0 loss : 1.2547118323189872 Acc : 28.703703703703702%

Epoch: 0
{0: 11, 1: 12, 2: 13}
Test epoch : 0 loss : 1.0878044366836548 Acc : 30.555555555555557%
30.555555555555557

Epoch: 1
Train epoch : 1 loss : 1.1438394274030412 Acc : 33.333333333333336%

Epoch: 1
{0: 11, 1: 12, 2: 13}
Test epoch : 1 loss : 1.1838473081588745 Acc : 30.555555555555557%
30.555555555555557

Epoch: 2
Train epoch : 2 loss : 1.214103443281991 Acc : 31.48148148148148%

Epoch: 2
{0: 11, 1: 12, 2: 13}
Test epoch : 2 loss : 1.1755651235580444 Acc : 30.555555555555557%
30.555555555555557

Epoch: 3
Train epoch : 3 loss : 1.1819052696228027 Acc : 37.96296296296296%

Epoch: 3
{0: 11, 1: 12, 2: 13}
Test epoch : 3 loss : 1.196570912996928 Acc : 30.555555555555557%
30.555555555555557

Epoch: 4
Train epoch : 4 loss : 1.1365218332835607 Acc : 31.48148148148148%

Epoch: 4
{0: 11, 1: 12, 2: 13}
Test epoch : 4 loss : 1.2075746456782024 Acc : 30.555555555555557%
30.555555555555557

Epoch: 5
Train epoch : 5

In [11]:
# 테스트셋에서 평가
model.load_state_dict(torch.load(os.path.join(save_path, f'teacher.pth')))
test(-1, testloader, 'test')


Epoch: -1
{0: 13, 1: 11, 2: 12}
Test epoch : -1 loss : 0.6049990256627401 Acc : 69.44444444444444%
